### LangGraph Basics Overview

In [1]:
!pip install langgraph langchain anthropic langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 911.6 kB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29


In [2]:
import sys , os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Get API Keys

In [9]:
sys.path.append("/content/drive/MyDrive/Colab Notebooks/")
from api_keys import _api_keys

#environment variables
os.environ["ANTHROPIC_API_KEY"] = _api_keys["ANTHROPIC_API_KEY"]
os.environ["LANGSMITH_API_KEY"] = _api_keys["LANGSMITH_API_KEY"]
!export LANGSMITH_TRACING=true

#model info
ANTHROPIC_MODEL='claude-3-haiku-20240307'

### START HERE
- https://langchain-ai.github.io/langgraph/#example

In [6]:
from typing import Literal
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [7]:
# Define the tools for the agent to use
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


In [10]:
#tools available for use
tools = [search]

#TOOL NODE
#This node acts as a bridge between the language model and the available tools.
#The state MUST contain a list of messages.
#The last message MUST be an `AIMessage`.
#The `AIMessage` MUST have `tool_calls` populated.
tool_node = ToolNode(tools)

#model to use in langraph calls
#bind_tools() ensures the model knows that is has tools available
model = ChatAnthropic(model=ANTHROPIC_MODEL
, temperature=0).bind_tools(tools)


In [ ]:
# Define the function that determines whether to call a tool or return a message to the user
# aka determines weather to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:

    #get last message
    messages = state['messages']
    last_message = messages[-1]

    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"

    # Otherwise, we stop (reply to the user)
    return END

# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}